In [1]:
!python -m spacy download pt_core_news_sm

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-05 19:39:20.109371: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 58.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [13]:
import spacy
import numpy as np
from gensim.models import KeyedVectors
import pandas as pd

w2v_modelo_cbow = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/Word2Vec/modelo_cbow.txt")
w2v_modelo_sg = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/Word2Vec/modelo_skip.txt")

dados_treino = pd.read_csv('https://caelum-online-public.s3.amazonaws.com/1638-word-embedding/treino.csv')
dados_teste = pd.read_csv('https://caelum-online-public.s3.amazonaws.com/1638-word-embedding/teste.csv')

In [7]:
nlp = spacy.load("pt_core_news_sm", disable=["paser", "ner", "tagger", "textcat"])
def tokenizador(texto):
  doc = nlp(texto)
  tokens_validos = []

  for token in doc:
    eh_valido = not token.is_stop and token.is_alpha
    if eh_valido:
      tokens_validos.append(token.text.lower())

  return tokens_validos

texto = "Fortaleza é uma cidade $$$$ 55555 ##### incrível!!!"
tokens = tokenizador(texto)

print(tokens)

['fortaleza', 'cidade', 'incrível']


In [12]:
def combinacao_vetores_por_soma(palavras, modelo):
  vetor_resultante = np.zeros(300)

  for pn in palavras:
    try:
      vetor_resultante += modelo.get_vector(pn)
    except KeyError:
      pass
  return vetor_resultante

vetor_texto = combinacao_vetores_por_soma(tokens, w2v_modelo_cbow)
print(vetor_texto.shape)
print(vetor_texto)

(300,)
[-0.41248731 -0.69784693  0.24261412  1.35988978  1.17959978 -0.19188082
 -0.14235481  0.16240515 -0.7801791  -0.02844487 -0.60636262 -0.19565268
  0.96479266  0.72234389  1.12243676  2.19772688  0.40561893  0.28030168
 -2.00362372  0.07078053 -1.15371765  1.18468623 -1.11321665  1.58370996
 -0.15997236  1.97695974  0.759952    1.76944063 -0.36425824  0.55207171
  0.86069857  1.34679293 -0.01148035  0.25165271  0.31117812  0.08389964
 -0.9710695   0.46957761 -0.68102858  0.45465656 -0.8321553  -0.70454323
  1.45618498 -0.23971074 -0.87767038  0.5276929   1.66046585 -0.84398486
 -0.80878198 -1.02992272  0.12113244 -1.98247769 -0.54525698 -0.35189296
  0.39910114  0.30594108  0.526636    0.66826106 -0.78611574 -0.55135212
  1.6701844   1.04422495  0.25188231  1.87358974 -0.37380314 -0.25829713
 -0.64687449  0.49873542 -1.00642863 -0.03128857  0.89258738  1.66749043
  1.04680514 -1.28668229  0.9833913   2.66784656  1.81418898 -0.75354861
 -0.11334311  0.93812709  0.97986634 -1.1623

In [14]:
def matriz_vetores(textos, modelo):
  x = len(textos)
  y = 300
  matriz = np.zeros((x,y))

  for i in range(x):
    palavras = tokenizador(textos.iloc[i])
    matriz[i] = combinacao_vetores_por_soma(palavras, modelo)

  return matriz


matriz_vetores_treino_cbow = matriz_vetores(dados_treino.title, w2v_modelo_cbow)
matriz_vetores_teste_cbow = matriz_vetores(dados_teste.title, w2v_modelo_cbow)

print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape)

(90000, 300)
(20513, 300)


In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classificador(modelo, x_treino, y_treino, x_teste, y_teste):
  RL = LogisticRegression(max_iter=800)
  RL.fit(x_treino, y_treino)
  categorias = RL.predict(x_teste)
 
  resultados = classification_report(y_teste, categorias)
  print(resultados)
  return RL
RL_cbow = classificador(w2v_modelo_cbow,
        matriz_vetores_treino_cbow,
        dados_treino.category,
        matriz_vetores_teste_cbow,
        dados_teste.category)

              precision    recall  f1-score   support

     colunas       0.80      0.71      0.75      6103
   cotidiano       0.63      0.80      0.71      1698
     esporte       0.93      0.86      0.89      4663
   ilustrada       0.13      0.85      0.23       131
     mercado       0.83      0.78      0.81      5867
       mundo       0.73      0.83      0.78      2051

    accuracy                           0.78     20513
   macro avg       0.68      0.80      0.69     20513
weighted avg       0.81      0.78      0.80     20513



In [17]:
matriz_vetores_treino_sg = matriz_vetores(dados_treino.title, w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(dados_teste.title, w2v_modelo_sg)

print(matriz_vetores_treino_sg.shape)
print(matriz_vetores_teste_sg.shape)

RL_sg = classificador(w2v_modelo_sg,
        matriz_vetores_treino_sg,
        dados_treino.category,
        matriz_vetores_teste_sg,
        dados_teste.category)

(90000, 300)
(20513, 300)
              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.64      0.80      0.71      1698
     esporte       0.93      0.87      0.90      4663
   ilustrada       0.14      0.87      0.24       131
     mercado       0.84      0.79      0.82      5867
       mundo       0.75      0.84      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.69      0.82      0.70     20513
weighted avg       0.82      0.79      0.80     20513



In [18]:
import pickle 

with open("/content/drive/MyDrive/Word2Vec/RL_cbow.pkl", "wb") as f:
  pickle.dump(RL_cbow, f)

with open("/content/drive/MyDrive/Word2Vec/RL_sg.pkl", "wb") as f:
  pickle.dump(RL_sg, f)